In [1]:
import numpy as np
import pandas as pd # для работы с DataFrame 

# загружаем датасет
column_names = ['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT', 'PRICE']
boston_data = pd.read_csv('housing.csv', header=None, delimiter=r"\s+", names=column_names)
boston_data.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,PRICE
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222.0,18.7,396.90,5.33,36.2


In [2]:
# формируем матрицу А из столбца единиц и факторов CRIM, RM + вектор целевой переменной 
CRIM = boston_data['CRIM']
RM = boston_data['RM']
A = np.column_stack((np.ones(506), CRIM, RM))
y = boston_data[['PRICE']]
print(A)

[[1.0000e+00 6.3200e-03 6.5750e+00]
 [1.0000e+00 2.7310e-02 6.4210e+00]
 [1.0000e+00 2.7290e-02 7.1850e+00]
 ...
 [1.0000e+00 6.0760e-02 6.9760e+00]
 [1.0000e+00 1.0959e-01 6.7940e+00]
 [1.0000e+00 4.7410e-02 6.0300e+00]]


In [3]:
A.shape

(506, 3)

In [4]:
w_hat = np.linalg.inv(A.T@A)@A.T@y
w_hat.values

array([[-29.24471945],
       [ -0.26491325],
       [  8.39106825]])

In [11]:
# составим прогноз для нашей модели
# добавились новые данные 
CRIM_new = 0.1
RM_new = 8
# делаем прогноз типичной стоимости дома
PRICE_new = w_hat.iloc[0]+w_hat.iloc[1]*CRIM_new+w_hat.iloc[2]*RM_new
PRICE_new.values

array([37.85733519])

In [12]:
# короткий способ сделать прогноз
new = np.array([[1, CRIM_new, RM_new]])
print('prediction:', (new@w_hat).values)

prediction: [[37.85733519]]


In [8]:
from sklearn.linear_model import LinearRegression
# создаем модель линейной регрессии
# указываем fit_intercept=False, тк в матрице А и так есть столбец с единицами
model = LinearRegression(fit_intercept=False)
# вычисляем коэффициенты регрессии
# передаем в fit матрицу А и вектор целевой переменной
model.fit(A,y)
print('w_hat:', model.coef_)
new_prediction = model.predict(new)
print('prediction:', new_prediction)


w_hat: [[-29.24471945  -0.26491325   8.39106825]]
prediction: [[37.85733519]]


In [13]:
# создадим вырожденную матрицу А
A = np.array([
    [1, 1, 1, 1], 
    [2, 1, 1, 2], 
    [-2, -1, -1, -2]]
).T
y = np.array([1, 2, 5, 1])
# вычислим OLS-оценку для коэффициентов
w_hat=np.linalg.inv(A.T@A)@A.T@y
print(w_hat) 
## LinAlgError: Singular matrix

LinAlgError: Singular matrix

In [14]:
# создаём модель линейной регрессии
model = LinearRegression(fit_intercept=False)
# вычисляем коэффициенты регрессии
model.fit(A, y)
print('w_hat:', model.coef_)
## w_hat: [ 6.   -1.25  1.25]

w_hat: [ 6.   -1.25  1.25]


Это происходит, потому что в sklearn предусмотрена борьба с плохо определёнными (близкими к вырожденным и вырожденными) матрицами.

Для этого используется метод под названием сингулярное разложение (SVD).

Суть метода заключается в том, что в OLS-формуле мы на самом деле используем не саму матрицу А, а её диагональное представление из сингулярного разложения, которое гарантированно является невырожденным.

In [15]:
# классическая OLS-регрессия в numpy с возможностью получения решения даже для вырожденных матриц
np.linalg.lstsq(A, y, rcond=None)

(array([ 6.  , -1.25,  1.25]),
 array([], dtype=float64),
 np.int32(2),
 array([4.86435029, 0.58146041, 0.        ]))

Функция возвращает четыре значения:

1. вектор рассчитанных коэффициентов линейной регрессии;

2. сумму квадратов ошибок, MSE (она не считается, если ранг матрицы меньше числа неизвестных, как в нашем случае);

3. ранг матрицы ;

4. вектор из сингулярных значений, которые как раз и оберегают нас от ошибки (о них мы поговорим позже).